# Speech2Text

**Information**

The Speech2Text model of OpenAI was proposed in the article [Robust Speech Recognition via Large-Scale Weak Supervision](https://arxiv.org/abs/2212.04356)

The **Whisper large-v3-turbo model** is a streamlined, faster version of OpenAI's Whisper large-v3, reducing decoding layers from 32 to 4 to improve speed with minimal quality loss. Trained on over 5 million hours of labeled data, this model is robust in zero-shot speech recognition and translation across diverse datasets and domains.


Remark: Model is based on [openai/whisper-large-v3](https://huggingface.co/openai/whisper-large-v3), which was developed by OpenAI

***
**Coding sources**

* Hugging Face model page: https://huggingface.co/openai/whisper-large-v3-turbo
* Hugging Face documentation: https://huggingface.co/docs/transformers/main/en/model_doc/whisper


***
**Aim of the code template**

Mimic the Advanced Speech-to-Text API of Google by i. generating an audio file (Text2Speech), ii. transcribing the audio file (Speech2Text) and iii. improve the transcription by using two LLMs; see Google API: https://cloud.google.com/speech-to-text/?hl=en

# Transcripe your audio file

In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("device:", device)

torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
    # generate_kwargs={"task": "translate", "language": "german"} # translate the audio file, else whisper predicts the language of the source audio automatically and the source audio language is the same as the target text language.
)

# Load audio data from your local "dialog_suno.wav" file
audio_file = "../Text2Speech/dialog_suno.wav"
audio_input, sample_rate = sf.read(audio_file)

# Process and transcribe the audio data
result = pipe({"array": audio_input, "sampling_rate": sample_rate})
print(result["text"])

device: cpu


/home/fenn/Documents/env_python/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


 So, let's brainstorm potential applications for large language models. With their ability to process and generate human-like text, there are tons of possibilities. What comes to mind first? Definitely customer support. LLMs can handle a large volume of basic inquiries like troubleshooting and FAQs 24-7. This would free up human agents to focus on more complex issues. Agreed. They'd also be great for content creation. Think of generating marketing copy, blogs, or even personalized emails. It could save so much time and maintain brand voice consistent. Right. And education too. LLM's could serve as tutors, explaining concepts in various ways until a student understands, interactive, and responsive learning. Another area is healthcare. They could assist in medical documentation or patient prescreening, which could speed up processes in busy clinics. Also research, you know, analyzing large data sets, summarizing reports, or even helping draft papers. Researchers would save hours. Exactly

# Improve the transcription of your audio file

## Environment Setup

Get API key(s):

In [2]:
import os
import sys

# Assuming 'src' is one level down (in the current directory or a subdirectory)
path_to_src = os.path.join('../..','src')  # Moves one level down to 'src' folder

# Add the path to sys.path
sys.path.append(path_to_src)

# Now you can import your API_key module
import API_key as key

Code to improve your transcriped audio file:

In [3]:
from huggingface_hub import InferenceClient
import textwrap

transcription = result["text"]


# Initialize client
client = InferenceClient(token=key.hugging_api_key)

# Create prompts
system_content = "You are a helpful assistant, whos task is to improve the transcription of an audio file. Focus here on possible spelling errors and missing words."
user_content = f"""
    Check the transcription of the following audio file. Only provide the improved transcription:
    
    {transcription}
"""

# Feed prompts into model
output = client.chat_completion(
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ],
   model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    max_tokens=500,
    stream=False,
    temperature=0.0,
)


# Accessing the text in the output object
text = output.choices[0].message.content

# Printing the output in a more readable format
print('\n'.join(textwrap.wrap(text, 100)))

So, let's brainstorm potential applications for large language models. With their ability to process
and generate human-like text, there are tons of possibilities. What comes to mind first? Definitely
customer support. LLMs can handle a large volume of basic inquiries, like troubleshooting and FAQs,
24/7. This would free up human agents to focus on more complex issues. Agreed. They'd also be great
for content creation. Think of generating marketing copy, blogs, or even personalized emails. It
could save so much time and maintain a brand voice consistently. Right. And education too. LLMs
could serve as tutors, explaining concepts in various ways until a student understands, providing
interactive and responsive learning. Another area is healthcare. They could assist in medical
documentation or patient prescreening, which could speed up processes in busy clinics. Also,
research - you know, analyzing large data sets, summarizing reports, or even helping draft papers.
Researchers would save

# Summarize the transcribed audio file


In [4]:
from huggingface_hub import InferenceClient
import textwrap


# Initialize client
client = InferenceClient(token=key.hugging_api_key)

# Create prompts
system_content = f"""
<Context>
You are a helpful assistant, whos task is to summarize transcribed audio files. 
</Context>

<Task>
Please provide two lists:.
- Summarize the content regarding the most important points of discussion, be scientific
- Provide a list of open tasks, which need to be done.

Provide the second list only if there are any important tasks mentioned.
</Task>


<Data Structure>
Provide the lists in the following format:

List 1:
\\n
\\n
List 2:
</Data Structure>
"""
user_content = f"""
    Provide the two lists regarding the following text:
    {text}
"""

# Feed prompts into model
output = client.chat_completion(
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ],
   model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    max_tokens=500,
    stream=False,
    temperature=0.0,
)


# Accessing the text in the output object
text = output.choices[0].message.content

# Printing the output in a more readable format
print(text)

List 1:
- Large language models (LLMs) can be applied to various industries due to their ability to process and generate human-like text.
- Potential applications include customer support, handling basic inquiries and freeing human agents for complex issues.
- LLMs can aid in content creation, such as generating marketing copy, blogs, and personalized emails, saving time and maintaining a consistent brand voice.
- In education, LLMs can serve as interactive tutors, explaining concepts in multiple ways for better student understanding.
- Healthcare applications include assisting in medical documentation and patient prescreening, speeding up processes in busy clinics.
- LLMs can also aid researchers in analyzing large data sets, summarizing reports, and drafting papers, saving time.
- Balancing productivity gains with ethical considerations, ensuring transparency and control, especially with sensitive information, is crucial.

List 2:
- Draft specific use cases for each sector (customer 